This is my first experiment in this competition. Whereas XGBoost is highly recommended I rather tried to see how far I can go with an NN (using Keras).

This is the basic model and with 250 epochs has an accuracy of 80% (really poor).

I'll continue for a few days researching how much I can optimize this model.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
print(os.listdir("../input/"))
print(os.listdir("../working/"))
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

['learn-together']
['__notebook__.ipynb', '__output__.json']
/kaggle/input/learn-together/test.csv
/kaggle/input/learn-together/train.csv
/kaggle/input/learn-together/sample_submission.csv


In [2]:
#load data, I had an issue with the data and hacked the data into the kernel
#dftrain=pd.read_csv('/kaggle/input/train.csv')
#dftest=pd.read_csv('/kaggle/input/test.csv')
dftrain=pd.read_csv('/kaggle/input/learn-together/train.csv')
dftest=pd.read_csv('/kaggle/input/learn-together/test.csv')

In [3]:
#file shape
print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

   Id  Elevation  Aspect  Slope  Horizontal_Distance_To_Hydrology  \
0   1       2596      51      3                               258   
1   2       2590      56      2                               212   
2   3       2804     139      9                               268   
3   4       2785     155     18                               242   
4   5       2595      45      2                               153   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                               0                              510   
1                              -6                              390   
2                              65                             3180   
3                             118                             3090   
4                              -1                              391   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  ...  Soil_Type32  \
0            221             232            148  ...            0   
1            220          

In [4]:
#features types
print(dftrain.dtypes)
print(dftest.dtypes)

Id                                    int64
Elevation                             int64
Aspect                                int64
Slope                                 int64
Horizontal_Distance_To_Hydrology      int64
Vertical_Distance_To_Hydrology        int64
Horizontal_Distance_To_Roadways       int64
Hillshade_9am                         int64
Hillshade_Noon                        int64
Hillshade_3pm                         int64
Horizontal_Distance_To_Fire_Points    int64
Wilderness_Area1                      int64
Wilderness_Area2                      int64
Wilderness_Area3                      int64
Wilderness_Area4                      int64
Soil_Type1                            int64
Soil_Type2                            int64
Soil_Type3                            int64
Soil_Type4                            int64
Soil_Type5                            int64
Soil_Type6                            int64
Soil_Type7                            int64
Soil_Type8                      

In [5]:
#validate files - nans per feature
print(dftrain.isnull().sum(axis = 0))
print(dftest.isnull().sum(axis = 0))

Id                                    0
Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Wilderness_Area2                      0
Wilderness_Area3                      0
Wilderness_Area4                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0


In [6]:
#validate files - no rows with all zeros
print(dftrain[dftrain.drop(['Id','Cover_Type'], axis=1).eq(0).all(1)].empty)
print(dftest[dftest.drop('Id', axis=1).eq(0).all(1)].empty)

True
True


In [7]:
#split train data in features and labels
y = dftrain.Cover_Type
x = dftrain.drop(['Id','Cover_Type'], axis=1)

# split test data in features and Ids
Ids = dftest.Id
x_predict = dftest.drop('Id', axis=1)

In [8]:
#force all types to float
x = x.astype(float)
x_predict = x_predict.astype(float)

In [9]:
#normalize features
#in the future it can be done more elegantly, for now just using the max min values of the data that we know
#x['Elevation']=(x['Elevation']-x['Elevation'].min())/(x['Elevation'].max()-x['Elevation'].min())                             
x['Elevation']=(x['Elevation']-1859)/(3858-1859)                             
x['Aspect']=x['Aspect']/360                      
x['Slope']=x['Slope']/66                      
x['Horizontal_Distance_To_Hydrology']=x['Horizontal_Distance_To_Hydrology']/1397                      
x['Vertical_Distance_To_Hydrology']=(x['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x['Horizontal_Distance_To_Roadways']=x['Horizontal_Distance_To_Roadways']/7117                      
x['Hillshade_9am']=x['Hillshade_9am']/254                      
x['Hillshade_Noon']=x['Hillshade_Noon']/254                      
x['Hillshade_3pm']=x['Hillshade_3pm']/254                      
x['Horizontal_Distance_To_Fire_Points']=x['Horizontal_Distance_To_Fire_Points']/67173                      
                                
x_predict['Elevation']=(x_predict['Elevation']-1859)/(3858-1859)                             
x_predict['Aspect']=x_predict['Aspect']/360                      
x_predict['Slope']=x_predict['Slope']/66                      
x_predict['Horizontal_Distance_To_Hydrology']=x_predict['Horizontal_Distance_To_Hydrology']/1397                      
x_predict['Vertical_Distance_To_Hydrology']=(x_predict['Vertical_Distance_To_Hydrology']+173)/(601+173)                             
x_predict['Horizontal_Distance_To_Roadways']=x_predict['Horizontal_Distance_To_Roadways']/7117                      
x_predict['Hillshade_9am']=x_predict['Hillshade_9am']/254                      
x_predict['Hillshade_Noon']=x_predict['Hillshade_Noon']/254                      
x_predict['Hillshade_3pm']=x_predict['Hillshade_3pm']/254                      
x_predict['Horizontal_Distance_To_Fire_Points']=x_predict['Horizontal_Distance_To_Fire_Points']/67173                      

In [10]:
#validate data - no rows with all zeros
#x.index[x.eq(0).all(1)]
print(x[x.eq(0).all(1)].empty)
print(x_predict[x_predict.eq(0).all(1)].empty)

True
True


In [11]:
# convert the label to One Hot Encoding
#to_categorical requires 0..6 instead of 1..7
y -=1
y = y.to_numpy()

num_classes = 7

from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [12]:
#convert the features dataframes to numpy arrays
x = x.to_numpy()
x_predict = x_predict.to_numpy()

In [13]:
#split in train (80%) and test (20%) sets 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [14]:
#here is the NN model
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

num_features = 54

model = Sequential()
model.add(Dense(units=num_features, activation='relu', kernel_initializer='normal', input_dim=num_features))
model.add(Dense(units=num_features*2, activation='relu', kernel_initializer='normal'))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
#              optimizer='Adadelta',
              optimizer='Adam',
              metrics=['accuracy'])

Using TensorFlow backend.


In [15]:
#train the model
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=250)

Train on 12096 samples, validate on 3024 samples
Epoch 1/250
12096/12096 [==============================] - 1s 80us/sample - loss: 1.1389 - acc: 0.5398 - val_loss: 0.8916 - val_acc: 0.6002
Epoch 2/250
12096/12096 [==============================] - 1s 57us/sample - loss: 0.8597 - acc: 0.6290 - val_loss: 0.8233 - val_acc: 0.6495
Epoch 3/250
12096/12096 [==============================] - 1s 58us/sample - loss: 0.8043 - acc: 0.6565 - val_loss: 0.7869 - val_acc: 0.6551
Epoch 4/250
12096/12096 [==============================] - 1s 56us/sample - loss: 0.7688 - acc: 0.6711 - val_loss: 0.7429 - val_acc: 0.6852
Epoch 5/250
12096/12096 [==============================] - 1s 58us/sample - loss: 0.7372 - acc: 0.6835 - val_loss: 0.7162 - val_acc: 0.6994
Epoch 6/250
12096/12096 [==============================] - 1s 58us/sample - loss: 0.7134 - acc: 0.6939 - val_loss: 0.7009 - val_acc: 0.7017
Epoch 7/250
12096/12096 [==============================] - 1s 57us/sample - loss: 0.6960 - acc: 0.7039 - val_lo

In [16]:
# Predict!!
y_predict = model.predict(x_predict)

In [17]:
for i in range(10):
	print(y_predict[i], np.argmax(y_predict[i])+1)

[8.8794880e-02 9.0312362e-01 1.4086723e-12 1.6156637e-16 4.3548578e-03
 3.7266195e-03 5.8637487e-24] 2
[5.5046115e-02 6.3986635e-01 7.6340850e-10 3.9300442e-13 3.0329975e-01
 1.7877951e-03 3.1392076e-21] 2
[5.1251240e-02 8.9108670e-01 3.0888173e-10 1.3277933e-12 5.6322265e-02
 1.3398563e-03 1.8855041e-21] 2
[3.6933254e-02 9.4412106e-01 1.1578282e-10 6.5304381e-12 1.6752902e-02
 2.1927692e-03 1.0996842e-21] 2
[2.6851648e-02 9.6310312e-01 6.6268248e-11 4.5178784e-11 6.0077095e-03
 4.0375651e-03 3.6524639e-22] 2
[3.58752310e-02 9.45886731e-01 1.11850154e-10 2.73890871e-12
 1.42911077e-02 3.94684961e-03 1.71135649e-22] 2
[4.7139239e-02 9.2674136e-01 1.3099853e-10 8.3363633e-13 2.2467950e-02
 3.6514436e-03 5.3641249e-22] 2
[4.5815326e-02 8.9776725e-01 2.3710850e-10 5.4704033e-13 5.3527337e-02
 2.8900309e-03 5.3785152e-22] 2
[8.9630134e-02 7.7242947e-01 1.1855985e-09 2.2447266e-13 1.3503878e-01
 2.9016202e-03 1.0445658e-21] 2
[6.6966169e-02 8.5741240e-01 3.7254697e-10 4.4693378e-13 7.1911111

In [18]:
# Save predictions to a file for submission
#argmax give us the highest probable label
# we add one to the predictions to scale from 0..6 to 1..7
output = pd.DataFrame({'Id': Ids,
                       'Cover_Type': y_predict.argmax(axis=1)+1})
output.to_csv('submission.csv', index=False)

In [19]:
#create a link to download the file    
from IPython.display import FileLink
FileLink(r'submission.csv')

/kaggle/working/submission.csv